In [ ]:
#This is a simple helper function to help unpack the nested data in the train.csv file for the MLB digital engagement contest.
#The pandas code isn't optimized, it uses iterrows() that is slow and we could have been more efficient with data processing

In [ ]:
import os
import pandas as pd

train_file = os.path.join("/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv")

train_df = pd.read_csv(train_file)

train_df.head()

In [ ]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
import sqlite3
import os

def process_nested_df_column_and_add_to_sqlite(column_header):
    
    unnested_df = pd.DataFrame()
    rows_processed = 0

    for i, row in train_df.iterrows():
        entry_df = unpack_json(row[column_header])
        if unnested_df.shape[0] == 0:
            unnested_df = entry_df
        else:
            if entry_df.shape[1] > 0:
                assert unnested_df.shape[1] == entry_df.shape[1]
                assert sorted(unnested_df.columns.values) == sorted(entry_df.columns.values)
                unnested_df = unnested_df.append(entry_df)
        rows_processed += 1

    print("rows_processed", column_header, rows_processed)
    
    dest_db = os.path.join("/kaggle/working/mlb_features.db")

    conn = sqlite3.connect(dest_db)
    unnested_df.to_sql(column_header, conn, index=False)
    conn.close()

In [ ]:
column_headers = [c for c in train_df.columns.values if c != "date"]

for column_header in column_headers:
    print("Processing", column_header)
    process_nested_df_column_and_add_to_sqlite(column_header=column_header)

In [ ]:
test_file = os.path.join("/kaggle/input//mlb-player-digital-engagement-forecasting/example_test.csv")

test_df = pd.read_csv(test_file, nrows=3)

print(min(test_df["date"]), max(test_df["date"]))
test_df.head()